In [178]:
# Hindi_English_Word Classifcation

# Step1 : read the dataset(label)
# Step2 : Create word2Id dictionary for the ascii
#           {'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5....}
# Step 3: Create the Charters Matrix for the word Corpus
#        [['c', 'r', 'i', 'c', 'k', 'e', 't'],['c', 'h', 'a', 'l'],['r', 'a', 'h', 'a']]......
#Step 4: Convert word to respected Interger value(Sequence matrix)
#        [[3, 18, 9, 3, 11, 5, 20], [3, 8, 1, 12], [18, 1, 8, 1]]
#Step 5: Padding the Sequcne of max length = 30
#        array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
#                0,  0,  0,  0,  0,  0,  3, 18,  9,  3, 11,  5, 20])
#Step 6: Encode the target values {0:"English",1:"Hindi"}
#Step 7: Create Keras model alsog with LSTM layers
#Step 8: Reshape the X_doc_vec to 3D
#        X_train=X_train.reshape(10199,30,1)
#Step 9: Crate Prediction function which reads and convert to respected vector form for prediction

In [310]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

data = pd.read_csv(r"New_Hinglish_Classification_final_1.csv")

In [179]:
data.shape

(12749, 3)

In [180]:
data.head(3)

,english,Word,label
0,0,cricket,en
1,1,chal,hi
2,2,raha,hi


In [181]:
data['label'] = data['label'].map({'en':"English",'hi':"Hindi"})

data = data[['Word','label']]

In [182]:
data['label_encoded'] = data['label'].map({"English":0,"Hindi":1})

In [183]:
data.head()

,Word,label,label_encoded
0,cricket,English,0
1,chal,Hindi,1
2,raha,Hindi,1
3,hain,Hindi,1
4,yaha,Hindi,1


In [184]:
data['label'].value_counts()

English    6935
Hindi      5814
Name: label, dtype: int64

In [185]:
data.isnull().sum()

Word             0
label            0
label_encoded    0
dtype: int64

In [186]:
# Create a dictionary for alphabets and integer value

from string import ascii_lowercase

od = dict(((x,y) for y,x in enumerate(ascii_lowercase,1)))

print(od)

{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}


In [187]:
# Preprocessing :Lower case the word

data['Word'].apply(lambda x : x.lower())

0          cricket
1             chal
2             raha
3             hain
4             yaha
           ...    
12744    challenge
12745      winners
12746           at
12747     paradise
12748       photos
Name: Word, Length: 12749, dtype: object

In [188]:
#list(data['label'])

word = [x for x in list(data['Word'])]
word_vec = [[char for char in s] for s in labels]

word_vec[:10]

[['c', 'r', 'i', 'c', 'k', 'e', 't'],
 ['c', 'h', 'a', 'l'],
 ['r', 'a', 'h', 'a'],
 ['h', 'a', 'i', 'n'],
 ['y', 'a', 'h', 'a'],
 ['i'],
 ['l', 'i', 'k', 'e', 'd'],
 ['a'],
 ['y', 'o', 'u', 't', 'u', 'b', 'e'],
 ['v', 'i', 'd', 'e', 'o']]

['E', 'n', 'g', 'l', 'i', 's', 'h']

In [237]:
def word2id(word):
    new_vec = []
    
    word = word.lower()
    
    vec = []
    for j in word:
        value = od[j]
        vec.append(value)
    new_vec.append(vec)
    
    return new_vec[0]    

In [236]:
word2id("English")

[5, 14, 7, 12, 9, 19, 8]

In [245]:
# Replace the char to repected integet value

word_vec[:2]

# Contins sequence Matrix
new_vec = []

for x in word_vec:
    vec = []
    for j in x:
        value = od[j]
        vec.append(value)
    new_vec.append(vec)
    
# Interger encoding of the word_vec
new_vec[:4]

[[3, 18, 9, 3, 11, 5, 20], [3, 8, 1, 12], [18, 1, 8, 1], [8, 1, 9, 14]]

In [190]:
# padding

from tensorflow.keras.preprocessing.sequence import pad_sequences
maxl = 30
x_data_vec = pad_sequences(new_vec, maxlen=maxl)

In [191]:
x_data_vec.shape # 12749 rows of data and sequence length of 30

(12749, 30)

In [192]:
x_data_vec[0]

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  3, 18,  9,  3, 11,  5, 20])

In [203]:
y_data_vec = np.array(data['label_encoded'])

y_data_vec.shape

(12749,)

In [202]:
x_data_vec.shape

(12749, 30)

In [204]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(x_data_vec, y_data_vec, test_size=0.20)

In [205]:
X_train.shape

(10199, 30)

In [206]:
X_test.shape

(2550, 30)

In [207]:
Y_train.shape

(10199,)

In [208]:
# Reshare the vectors for LSTM 

X_train=X_train.reshape(10199,30,1)
X_test=X_test.reshape(2550,30,1)
Y_train=Y_train.reshape(10199,1)
Y_test=Y_test.reshape(2550,1)

In [209]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout


model_classifier =Sequential()
model_classifier.add(LSTM(128, return_sequences = True, input_shape = (X_train.shape[1], 1)))
model_classifier.add(LSTM(64, activation='tanh'))
model_classifier.add(Dense(1, activation='sigmoid'))
#model_classifier.add(Dense(2, activation='softmax'))
model_classifier.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Using TensorFlow backend.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 30, 128)           66560     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 116,033
Trainable params: 116,033
Non-trainable params: 0
_________________________________________________________________


In [210]:
# try using different optimizers and different optimizer configs
model_classifier.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [211]:
model_classifier.fit(X_train, Y_train,batch_size=128,epochs=50,validation_data=(X_test, Y_test))


Train on 10199 samples, validate on 2550 samples
Epoch 1/50
10199/10199 [==============================] - 14s 1ms/step - loss: 0.6038 - accuracy: 0.6659 - val_loss: 0.5686 - val_accuracy: 0.7129
Epoch 2/50
10199/10199 [==============================] - 13s 1ms/step - loss: 0.5580 - accuracy: 0.7162 - val_loss: 0.5524 - val_accuracy: 0.7204
Epoch 3/50
10199/10199 [==============================] - 13s 1ms/step - loss: 0.5401 - accuracy: 0.7255 - val_loss: 0.5509 - val_accuracy: 0.7231
Epoch 4/50
10199/10199 [==============================] - 13s 1ms/step - loss: 0.5264 - accuracy: 0.7314 - val_loss: 0.5355 - val_accuracy: 0.7169
Epoch 5/50
10199/10199 [==============================] - 13s 1ms/step - loss: 0.5179 - accuracy: 0.7384 - val_loss: 0.5210 - val_accuracy: 0.7208
Epoch 6/50
10199/10199 [==============================] - 13s 1ms/step - loss: 0.4994 - accuracy: 0.7500 - val_loss: 0.5199 - val_accuracy: 0.7341
Epoch 7/50
10199/10199 [==============================] - 13s 1ms/st

In [255]:
pad_sequences([word2id("english")], maxlen=maxl).shape

(1, 30)

In [ ]:


X_train.reshape(10199,30,1)

In [304]:
dict = {0:"English",1:"Hindi"}

'''Method to preidct the Language'''
def predictLanguage(model,word):
    
    # Word 2 vector conversion
    word_vec = pad_sequences([word2id(word)], maxlen=maxl)
    
    # Reshape the vector 3D
    word_vec = word_vec.reshape(1,30,1)
    
    # Predict the word from model
    pred = model.predict(word_vec)
    
    pred_output = pred[0,0]
    
    # Set the boundry
    if pred_output >= 0.5:
        output = 1
    else:
        output = 0
    
    # return thr output
    return dict[output]

In [305]:
predictLanguage(model_classifier,"kya")

'Hindi'

In [306]:
# choose a random element from a list
from random import seed
from random import choice
# seed random number generator
seed(1)
# prepare a sequence
sequence = [i for i in range(20)]
# make choices from the sequence

index = []

for _ in range(10):
    selection = choice(sequence)
    index.append(selection)
    
actual_output = []
predicted_output = []

df = data.iloc[index,:]

In [307]:
df

,Word,label,label_encoded
4,yaha,Hindi,1
18,cover,English,0
2,raha,Hindi,1
8,youtube,English,0
3,hain,Hindi,1
15,kishore,Hindi,1
14,hai,Hindi,1
15,kishore,Hindi,1
12,pe,Hindi,1
6,liked,English,0


In [308]:
for index, row in df.iterrows():
    
    actual_output.append(row['label'])
    
    word = row['Word']
    
    predicted_output.append(predictLanguage(model_classifier,word))


In [309]:
pd.DataFrame({"Actual":actual_output,"Predicted":predicted_output})

,Actual,Predicted
0,Hindi,Hindi
1,English,English
2,Hindi,Hindi
3,English,English
4,Hindi,Hindi
5,Hindi,English
6,Hindi,Hindi
7,Hindi,English
8,Hindi,Hindi
9,English,English
